<a href="https://colab.research.google.com/github/vladgap/LLC/blob/main/Rotem%20LLC%20simulation%2010.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from sklearn import preprocessing
import matplotlib.pyplot as plt

from scipy import optimize

In [ ]:
!pip install MLNN
from MLNN import*

In [ ]:
!wget -O LLC.py "https://raw.githubusercontent.com/vladgap/LLC/main/LLC_versions/LLC_5_1.py"
import LLC

In [ ]:
def CopyPasteToArray(a):
  if a.startswith('\n'):
    b=a[1:]
  else:
    b=a
  if b.endswith('\n'):
    b=b[:-1]
  c=b.replace('\t',',')
  d=c.split('\n')
  f=[]
  for e in d:
    if e.replace(',','').replace('.','').isdigit(): # only digits no letters
      f.append(list(eval(e)))
    else:
      f.append(e.split(','))
  return f

# EQUIL data

## EQUIL_rotem_lit1
lit data with NN model (7 hidden sigmoids)

### Equilibrium literature data

In [ ]:
data1=pd.DataFrame(data=[
[63.5,	1.3,	42.4,	15.0],
[58.7,	1.2,	36.9,	13.8],
[52.0, 1.0,	27.7,	11.8],
[45.5,	1.0,	18.9,	9.6],
[39.1,	1.4,	10.2,	8.3],
[31.5,	1.5,	5.1,	8.8],
[22.9,	2.0,	2.3,	9.0],
[12.6,	2.2,	0.90,	9.3],
[6.3,	2.4,	0.45,	9.45],
[0.01,	2.6,	0.01,	9.6],
],
columns=['H3PO4_aq', 'solvent_aq', 'H3PO4_org', 'water_org'])

In [ ]:
data1.head()

In [ ]:
fig1=go.Figure()
[fig1.add_trace(go.Scatter(x=data1.H3PO4_aq, y=data1[data1.columns[i]], name=data1.columns[i])) for i in range(1,4)]
fig1.update_layout(title='', width=1200, height=600,)
fig1.show()

In [ ]:
X1=np.array(data1)[:,[0]]
T1=np.array(data1)[:,[1,2,3]]

### Model
"Model for Rotem LLC simulation 1"

In [ ]:
scaler_X1 = preprocessing.StandardScaler()
scaler_T1 = preprocessing.StandardScaler()
scaler_X1.mean_, scaler_X1.scale_ = [33.211], [21.13387066]
scaler_T1.mean_, scaler_T1.scale_ = [1.66,  14.486, 10.465], [0.56071383, 15.23739033,  2.16703138]

bp1=VectorBackProp(layers=[1,7,3], hidden_activation = 'sigmoid')
manual_best_weights_for_now2=[[[-4.215621245080982,
   -8.235086763118087,
   -1.6251342150606451,
   -6.9206357615871035,
   -3.4092427169369826,
   -7.4263402914028775,
   -3.7003458539301977]],
 [[0.8764395040443358,
   3.9145850159876945,
   -1.5013585830931442,
   -1.194371163565281,
   4.118411550251139,
   5.892242255497829,
   -6.755593628941706]],
 [[-2.6609909047049918, -0.44480776958732143, 1.9256391063013463],
  [2.566022222516618, -0.5042582093932455, -1.432212975592254],
  [1.405004033981333, -0.2988952466228391, 0.5807591718257896],
  [2.0692859230668117, -0.0780871568942863, -0.5145373287488922],
  [-0.7989625307881565, -1.957459884661486, -2.8991426864622065],
  [-0.5821795598385298, -0.22284877970127598, -0.9077232625087237],
  [1.619488070885094, 0.03697105802710269, 0.008613201857731975]],
 [[-0.4010006364699687, 2.4687577407660806, 2.995012357373033]]]
bp1.import_weights(manual_best_weights_for_now2)

In [ ]:
class EQUIL_rotem_lit1: # literature data for pentanol 25C
# gets xout [%solvent_aq, %H3PO4_aq]
# calc xout , yout [%water_org, %H3PO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4], [%solv, %H3PO4], [%solv, %H3PO4], ...]
        # the model predicts %solvent_aq, %H3PO4_aq, %water_org from %H3PO4_aq
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        pred=scaler_T1.inverse_transform(bp1.run(scaler_X1.transform(x[:,[1]])))
        # prediction is column vector of lists [%solvent_aq, %H3PO4_aq, %water_org]

        self.xout = np.array([pred[:,0], x[:,1]]).T # [[%solvent_aq, %H3PO4_aq], ...]
        self.yout = np.array([pred[:,2], pred[:,1]]).T # [[%water_org, %H3PO4_org], ...]

## EQUIL_rotem_washing1
Rotem solvent, estimation by polynoms

In [ ]:
class EQUIL_rotem_washing1: # prepared using purified and washed solvents
# gets xout [%solvent_aq, %H3PO4_aq]
# calc xout , yout [%water_org, %H3PO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4], [%solv, %H3PO4], [%solv, %H3PO4], ...]
        y=np.zeros_like(x)
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # y = A + B*x + C*x^2 + D*x^3

        # %solvent_aq
        D,C,B,A = -3.78E-06,8.15E-04,- 5.73E-02,2.45E+00
        x[:,0] = A + B*x[:,1] + C*x[:,1]**2 + D*x[:,1]**3

        # %water_org
        D,C,B,A = 4.45E-05,- 1.56E-03,- 1.84E-02,7.58E+00
        y[:,0] = A + B*x[:,1] + C*x[:,1]**2 + D*x[:,1]**3

        # %H3PO4_org
        # D,C,B,A = 2.00E-04,- 4.86E-03,7.07E-02,2.73E-02
        # y[:,1] = A + B*x[:,1] + C*x[:,1]**2 + D*x[:,1]**3
        G,F,E,D,C,B,A = -1.97E-09,2.02E-07,- 2.95E-06,- 3.83E-05,2.60E-03,2.38E-02,1.70E-02
        y[:,1] = A + B*x[:,1] + C*x[:,1]**2 + D*x[:,1]**3 + E*x[:,1]**4 + F*x[:,1]**5 + G*x[:,1]**6

        self.xout = np.array([x[:,0], x[:,1]]).T # [[%solvent_aq, %H3PO4_aq], ...]
        self.yout = np.array([y[:,0], y[:,1]]).T # [[%water_org, %H3PO4_org], ...]

In [ ]:
# EQUIL_rotem_washing1([[2,20],[1.5,30],[1.3,40]]).xout

## EQUIL_rotem_washing2
Rotem solvent, full profile, sulfate

In [ ]:
data=CopyPasteToArray('''
H3PO4_aq	SO4_aq	H3PO4_org	water_org	solvent_aq	SO4_org
4.15	0.147	0.0733	9.6	2.5	0.0626
14.9	0.0806	0.359	9.5	2.2	0.0852
28.6	0.0329	1.68	9.3	1.9	0.0787
45.7	0.0106	8.31	8.5	1.6	0.0664
48.2	0.0078	10.7	8.5	1.3	0.0631
49.4	0.0058	12.2	8.7	1.0	0.0641
''')
# data
# data2=pd.DataFrame(data, columns=['H3PO4_aq',	'SO4_aq',	'H3PO4_org',	'water_org',	'solvent_aq',	'SO4_org'])
data2=pd.DataFrame(data[1:], columns=data[0])
data2

In [ ]:
X2=np.array(data2)[:,[0,1]]
T2_1=np.array(data2)[:,[2]]
T2_2=np.array(data2)[:,[3]]
T2_3=np.array(data2)[:,[4]]
T2_4=np.array(data2)[:,[5]]

scaler_X2 = preprocessing.StandardScaler().fit(X2)
scaler_T2_1 = preprocessing.StandardScaler().fit(T2_1)
scaler_T2_2 = preprocessing.StandardScaler().fit(T2_2)
scaler_T2_3 = preprocessing.StandardScaler().fit(T2_3)
scaler_T2_4 = preprocessing.StandardScaler().fit(T2_4)

X_sc2 = scaler_X2.transform(X2)
# T_sc2_1 = scaler_T2_1.transform(T2_1)
# T_sc2_2 = scaler_T2_1.transform(T2_2)
# T_sc2_3 = scaler_T2_1.transform(T2_3)
# T_sc2_4 = scaler_T2_1.transform(T2_4)

In [ ]:
class EQUIL_rotem_washing2: # from full profile, sulfate
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp2_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights2_1=[[[0.1735491429461148, 1.8140938189194906],
  [-0.6530409129171293, -0.6441814583921941]],
 [[0.7411897128053353, -1.2033881701371891]],
 [[0.3516552639162472], [1.6547118904069311]],
 [[-1.0860308848681413]]]
        bp2_1.import_weights(manual_weights2_1)
        predics2_1=abs(scaler_T2_1.inverse_transform(bp2_1.run(scaler_X2.transform(x[:,[1,2]]))))

        # water_org
        bp2_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights2_2=[[[-1.356083366626272, 2.1006456906712234],
  [-0.6309805208401018, -0.5374527149916659]],
 [[1.0515374304634164, -2.3182953246087115]],
 [[1.4841383017903245], [3.153523627853289]],
 [[-1.7074908699001965]]]
        bp2_2.import_weights(manual_weights2_2)
        predics2_2=abs(scaler_T2_2.inverse_transform(bp2_2.run(scaler_X2.transform(x[:,[1,2]]))))

        # solvent_aq
        bp2_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights2_3=[[[2.0994823528593702, -1.2293957752203057],
  [-0.6972915446623457, 0.5216771334283966]],
 [[-2.3459563115354527, 1.775004898861346]],
 [[-3.1726827033207408], [0.4025651395234753]],
 [[-0.4638947231253712]]]
        bp2_3.import_weights(manual_weights2_3)
        predics2_3=abs(scaler_T2_3.inverse_transform(bp2_3.run(scaler_X2.transform(x[:,[1,2]]))))

        # SO4_org
        bp2_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights2_4=[[[-2.7308241499437758, 1.8373933992350313],
  [-2.021898044039464, -0.8845457922101067]],
 [[1.0694417308325381, 0.08920200363835196]],
 [[2.7835653102552724], [1.6175935763910405]],
 [[-4.969235753353272]]]
        bp2_4.import_weights(manual_weights2_4)
        predics2_4=abs(scaler_T2_4.inverse_transform(bp2_4.run(scaler_X2.transform(x[:,[1,2]]))))

        self.xout = np.array([predics2_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics2_2[:,0], predics2_1[:,0], predics2_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

## EQUIL_rotem_washing3
ניסויי הפרדה עם ממס מטוהר

In [ ]:
Xscale=[21.97576471,  0.05575882], [15.2853469,   0.05047379]
Tscale=[1.89941176, 11.19176471,  1.4,         0.02921412], [2.40216868, 1.27085536, 0.17875386, 0.01997862]

scaler_X3 = preprocessing.StandardScaler().fit([[0,0]])
scaler_X3_2 = preprocessing.StandardScaler().fit([[0,0]]) # מים בקלה חושבו רק על נתונים "טובים
scaler_T3_1 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_2 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_3 = preprocessing.StandardScaler().fit([[0]])
scaler_T3_4 = preprocessing.StandardScaler().fit([[0]])

scaler_X3.mean_, scaler_X3.scale_ = Xscale[0], Xscale[1]
scaler_X3_2.mean_, scaler_X3_2.scale_ = [27.03608333,  0.0474    ], [14.68499343,  0.04213688] # מים בקלה חושבו רק על נתונים "טובים
scaler_T3_1.mean_, scaler_T3_1.scale_=[Tscale[0][0]], [Tscale[1][0]]
# scaler_T3_2.mean_, scaler_T3_2.scale_=[Tscale[0][1]], [Tscale[1][1]]
scaler_T3_2.mean_, scaler_T3_2.scale_=[10.81], [1.3091664] # מים בקלה חושבו רק על נתונים "טובים
scaler_T3_3.mean_, scaler_T3_3.scale_=[Tscale[0][2]], [Tscale[1][2]]
scaler_T3_4.mean_, scaler_T3_4.scale_=[Tscale[0][3]], [Tscale[1][3]]

In [ ]:
class EQUIL_rotem_washing3: # ניסויי הפרדה עם ממס מטוהר
# מודל מקורי
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp3_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_1=[[[-0.2328053114817587, 1.4197258515721494],
  [-0.013739923476318716, -0.009509632921841494]],
 [[1.5848282259786703, -0.9625030403646567]],
 [[-1.3007118041663503], [1.5725920350068667]],
 [[1.717]]]
        bp3_1.import_weights(manual_weights3_1)
        predics3_1=scaler_T3_1.inverse_transform(bp3_1.run(scaler_X3.transform(x[:,[1,2]])))

        # water_org
        bp3_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_2=[[[0.3909002105371126, -0.8406267181665068],
  [-0.4530958444599508, 0.704664998208326]],
 [[-0.09671769242746425, 0.7927475123043677]],
 [[0.48441213405704675], [-0.7676170242577324]],
 [[0.6435152247205708]]]
        bp3_2.import_weights(manual_weights3_2)
        predics3_2=scaler_T3_2.inverse_transform(bp3_2.run(scaler_X3_2.transform(x[:,[1,2]]))) # מים בקלה חושבו רק על נתונים "טובים

        # solvent_aq
        bp3_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_3=[[[0.6217022345240342, 2.8386999598609814],
  [-0.6968938452576019, 5.596033242601742]],
 [[2.7109590937198265, 0.22193377952697202]],
 [[-0.9372090292368469], [-0.33293100862641933]],
 [[3.277263990359097]]]
        bp3_3.import_weights(manual_weights3_3)
        predics3_3=scaler_T3_3.inverse_transform(bp3_3.run(scaler_X3.transform(x[:,[1,2]])))

        # SO4_org
        bp3_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_4=[[[-0.10168260750729177, 1.5507975795954547],
  [-0.02429351337425621, -5.794943503077378e-05]],
 [[1.5556257229985646, -0.978368082003519]],
 [[-1.3219817921544152], [1.6394490601340548]],
 [[1.5742747334295601]]]
        bp3_4.import_weights(manual_weights3_4)
        predics3_4=scaler_T3_4.inverse_transform(bp3_4.run(scaler_X3.transform(x[:,[1,2]])))

        self.xout = np.array([predics3_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics3_2[:,0], predics3_1[:,0], predics3_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

In [ ]:
class EQUIL_rotem_washing3_2: # ניסויי הפרדה עם ממס מטוהר
# מודל עם מים בקלה 12% גורף
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp3_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_1=[[[-0.2328053114817587, 1.4197258515721494],
  [-0.013739923476318716, -0.009509632921841494]],
 [[1.5848282259786703, -0.9625030403646567]],
 [[-1.3007118041663503], [1.5725920350068667]],
 [[1.717]]]
        bp3_1.import_weights(manual_weights3_1)
        predics3_1=abs(scaler_T3_1.inverse_transform(bp3_1.run(scaler_X3.transform(x[:,[1,2]]))))

        # water_org
        bp3_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_2=[[[0.3909002105371126, -0.8406267181665068],
  [-0.4530958444599508, 0.704664998208326]],
 [[-0.09671769242746425, 0.7927475123043677]],
 [[0.48441213405704675], [-0.7676170242577324]],
 [[0.6435152247205708]]]
        bp3_2.import_weights(manual_weights3_2)
        # predics3_2=abs(scaler_T3_2.inverse_transform(bp3_2.run(scaler_X3_2.transform(x[:,[1,2]])))) # מים בקלה חושבו רק על נתונים "טובים
        predics3_2=np.ones([len(x),1])*12 #  תוצאות אנליטיות לא מדויקות. לקחתי 12 באופן גורף

        # solvent_aq
        bp3_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_3=[[[0.6217022345240342, 2.8386999598609814],
  [-0.6968938452576019, 5.596033242601742]],
 [[2.7109590937198265, 0.22193377952697202]],
 [[-0.9372090292368469], [-0.33293100862641933]],
 [[3.277263990359097]]]
        bp3_3.import_weights(manual_weights3_3)
        predics3_3=abs(scaler_T3_3.inverse_transform(bp3_3.run(scaler_X3.transform(x[:,[1,2]]))))

        # SO4_org
        bp3_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_4=[[[-0.10168260750729177, 1.5507975795954547],
  [-0.02429351337425621, -5.794943503077378e-05]],
 [[1.5556257229985646, -0.978368082003519]],
 [[-1.3219817921544152], [1.6394490601340548]],
 [[1.5742747334295601]]]
        bp3_4.import_weights(manual_weights3_4)
        predics3_4=abs(scaler_T3_4.inverse_transform(bp3_4.run(scaler_X3.transform(x[:,[1,2]]))))

        self.xout = np.array([predics3_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics3_2[:,0], predics3_1[:,0], predics3_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

In [ ]:
class EQUIL_rotem_washing3_3: # ניסויי הפרדה עם ממס מטוהר
# מודל עם מים בקלה 12% גורף
# שינוי במשקלים של ממס בכבדה. שיהיה יותר אופקי
# gets xout [%solvent_aq, %H3PO4_aq, %SO4_aq]
# calc xout , yout [%water_org, %H3PO4_org, %SO4_org]

    def __init__(self, x):
        x=np.array(x, dtype=float) # [[%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], [%solv, %H3PO4, %SO4], ...]
        # the model predicts H3PO4_org, water_org ,solvent_aq, SO4_org
        # x[:,[1]] is column vector of %H3PO4_aq [[1], [2], ...]
        # x[:,[2]] is for %SO4_aq

        # H3PO4_org
        bp3_1=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_1=[[[-0.2328053114817587, 1.4197258515721494],
  [-0.013739923476318716, -0.009509632921841494]],
 [[1.5848282259786703, -0.9625030403646567]],
 [[-1.3007118041663503], [1.5725920350068667]],
 [[1.717]]]
        bp3_1.import_weights(manual_weights3_1)
        predics3_1=abs(scaler_T3_1.inverse_transform(bp3_1.run(scaler_X3.transform(x[:,[1,2]]))))

        # water_org
        bp3_2=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_2=[[[0.3909002105371126, -0.8406267181665068],
  [-0.4530958444599508, 0.704664998208326]],
 [[-0.09671769242746425, 0.7927475123043677]],
 [[0.48441213405704675], [-0.7676170242577324]],
 [[0.6435152247205708]]]
        bp3_2.import_weights(manual_weights3_2)
        # predics3_2=abs(scaler_T3_2.inverse_transform(bp3_2.run(scaler_X3_2.transform(x[:,[1,2]])))) # מים בקלה חושבו רק על נתונים "טובים
        predics3_2=np.ones([len(x),1])*12 #  תוצאות אנליטיות לא מדויקות. לקחתי 12 באופן גורף

        # solvent_aq
        bp3_3=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu') # modified solvent_aq weights
        manual_weights3_3=[[[-7.637254337641781, -0.490322880885377],
  [-1.8058063735873857, -0.10159866035465558]],
 [[-0.843126492292387, -2.3299686091075773]],
 [[0.2643116291427324], [0.4479958576488306]],
 [[-0.7056420231884761]]]
        bp3_3.import_weights(manual_weights3_3)
        predics3_3=abs(scaler_T3_3.inverse_transform(bp3_3.run(scaler_X3.transform(x[:,[1,2]]))))

        # SO4_org
        bp3_4=VectorBackProp(layers=[2,2,1], hidden_activation = 'prelu')
        manual_weights3_4=[[[-0.10168260750729177, 1.5507975795954547],
  [-0.02429351337425621, -5.794943503077378e-05]],
 [[1.5556257229985646, -0.978368082003519]],
 [[-1.3219817921544152], [1.6394490601340548]],
 [[1.5742747334295601]]]
        bp3_4.import_weights(manual_weights3_4)
        predics3_4=abs(scaler_T3_4.inverse_transform(bp3_4.run(scaler_X3.transform(x[:,[1,2]]))))

        self.xout = np.array([predics3_3[:,0], x[:,1], x[:,2]]).T # [[%solvent_aq, %H3PO4_aq, %SO4_aq], ...]
        self.yout = np.array([predics3_2[:,0], predics3_1[:,0], predics3_4[:,0]]).T # [[%water_org, %H3PO4_org, %SO4_org], ...]

        # print (self.yout, self.xout)

# Cases

## Literature data

### 1 - Washing



In [ ]:
bat1_2=LLC.Battery(stages_num=5, Oin=90, Ain=29, yin=[5,12], xin=[5,0], EQUIL=EQUIL_rotem_lit1, convergence=5e-4, eff=[1, 0.7], entrainment_perc_in=[2,5], entrainment_comp_in=[[5,5],[10,10]], entrainment_perc_out=[3,4])
# bat1_2=LLC.Battery(4, 87.6, 112.4, [0, 0], [0, 32.56], EQUIL=EQUIL_rotem_lit1, convergence=5e-4, eff=.7, entrainment_perc_out=10)

In [ ]:
bat1_2.runs

In [ ]:
table1_2=LLC.BatteryTableFull(bat1_2).data

In [ ]:
bat=bat1_2
fig1_3 = go.Figure()
n=1 #H3PO4
fig1_3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig1_3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig1_3.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig1_3.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig1_3.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig1_3.update_layout(title='', width=1200, height=600,)
fig1_3.show()

In [ ]:
fig1_4=go.Figure()
current_data = data1
current_table = table1_2
[fig1_4.add_trace(go.Scatter(x=current_data.H3PO4_aq, y=current_data[current_data.columns[i]], name=current_data.columns[i])) for i in range(1,4)]
fig1_4.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='x1 out, %'].values[0]))
fig1_4.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y2 out, %'].values[0]))
fig1_4.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y1 out, %'].values[0]))

# fig1_4.update_xaxes(title_text="x, %", range=[20, 40])
# fig1_4.update_yaxes(title_text="y, %", range=[0, 15])
fig1_4.update_layout(title='', width=1200, height=600,)
fig1_4.show()

### 2 - Extraction

In [ ]:
bat2=LLC.Battery(stages_num=7, Oin=90, Ain=29, yin=[5,0], xin=[0,40], EQUIL=EQUIL_rotem_lit1, convergence=5e-4, eff=[[.8, 1], [0.5, .8], [.6,.9]])

In [ ]:
bat2.runs

In [ ]:
bat2.eff

In [ ]:
table2=LLC.BatteryTableFull(bat2).data

In [ ]:
bat=bat2
fig2_1 = go.Figure()
n=1 #H3PO4
fig2_1.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig2_1.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig2_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig2_1.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig2_1.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig2_1.update_layout(title='', width=1200, height=600,)
fig2_1.show()

In [ ]:
fig2_2=go.Figure()
current_data = data1
current_table = table2
[fig2_2.add_trace(go.Scatter(x=current_data.H3PO4_aq, y=current_data[current_data.columns[i]], name=current_data.columns[i])) for i in range(1,4)]
fig2_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='x1 out, %'].values[0]))
fig2_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y2 out, %'].values[0]))
fig2_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y1 out, %'].values[0]))

# fig2_2.update_xaxes(title_text="x, %", range=[20, 40])
# fig2_2.update_yaxes(title_text="y, %", range=[0, 15])
fig2_2.update_layout(title='', width=1200, height=600,)
fig2_2.show()

## Washing data
Rotem solvent, estimation by polynoms

### 3 - Washing1

In [ ]:
bat3=LLC.Battery(stages_num=7, Oin=204, Ain=33.3, yin=[7.7,14.4], xin=[0,0], EQUIL=EQUIL_rotem_washing1)#, convergence=5e-4, eff=[1, 0.7], entrainment_perc_in=[2,5], entrainment_comp_in=[[5,5],[10,10]], entrainment_perc_out=[3,4])

In [ ]:
bat3.runs

In [ ]:
table3=LLC.BatteryTable(bat3).data

In [ ]:
bat=bat3
fig3 = go.Figure()
n=1 #H3PO4
fig3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig3.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig3.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig3.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig3.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig3.update_layout(title='', width=1200, height=600,)
fig3.show()

In [ ]:
fig3_2=go.Figure()
# current_data = data1
current_table = table3
# [fig1_4.add_trace(go.Scatter(x=current_data.H3PO4_aq, y=current_data[current_data.columns[i]], name=current_data.columns[i])) for i in range(1,4)]
fig3_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='x1 out, %'].values[0]))
fig3_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y2 out, %'].values[0]))
fig3_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y1 out, %'].values[0]))

# fig3_2.update_xaxes(title_text="x, %", range=[20, 40])
# fig3_2.update_yaxes(title_text="y, %", range=[0, 15])
fig3_2.update_layout(title='', width=1200, height=600,)
fig3_2.show()

### 4 - Washing2

In [ ]:
bat4=LLC.Battery(stages_num=7, Oin=204, Ain=33.3, yin=[7.7,14.4], xin=[0,0], EQUIL=EQUIL_rotem_washing1, convergence=5e-4, eff=[1, 0.9], entrainment_perc_in=[2,0], entrainment_comp_in=[[5,5],[0,0]], entrainment_perc_out=[3,4])

In [ ]:
bat4.runs

In [ ]:
table4=LLC.BatteryTableFull(bat4).data

In [ ]:
bat=bat4
fig4 = go.Figure()
n=1 #H3PO4
fig4.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig4.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig4.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig4.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig4.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig4.update_layout(title='Operation Data', width=1200, height=600,)
fig4.show()

In [ ]:
fig4_2=go.Figure()
# current_data = data1
current_table = table4
fig4_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='x1 out, %'].values[0]))
fig4_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y2 out, %'].values[0]))
fig4_2.add_trace(go.Scatter(x=current_table.loc[current_table.index=='x2 out, %'].values[0], y=current_table.loc[current_table.index=='y1 out, %'].values[0]))

# fig4_2.update_xaxes(title_text="x, %", range=[20, 40])
# fig4_2.update_yaxes(title_text="y, %", range=[0, 15])
fig4_2.update_layout(title='Equilibrium Data', width=1200, height=600,)
fig4_2.show()

## Rotem washing with sulfate

### 5 - Washing1

In [ ]:
EQUIL_rotem_washing2([[2.2,14.9,.0806]]).yout

In [ ]:
test1 = LLC.Stage(90, 29, [10, 14, .07], [2, 0.1, .02], EQUIL_rotem_washing2, eff=1)#, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10],[5,5]], convergence=1E-4)

In [ ]:
test1.runs

In [ ]:
test1.output()

In [ ]:
test1 = LLC.Stage(90, 29, [8.60822308, 0.09877921, 0.02661528], [2.06758444, 28.21705147, 0.11005343], EQUIL_rotem_washing2, eff=1)#, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10],[5,5]], convergence=1E-4)

In [ ]:
bat5_1=LLC.Battery(stages_num=4, Oin=90, Ain=29, yin=[8.60822308, 1, 0.05], xin=[2.06758444, 30, 0.3], EQUIL=EQUIL_rotem_washing2)

In [ ]:
bat5_1.runs

In [ ]:
table5=LLC.BatteryTable(bat5_1).data

In [ ]:
bat=bat5_1
fig5 = go.Figure()
n=1 #H3PO4
fig5.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_tag_list[:,n], name='Equilibrium', marker_color='black'))
fig5.add_trace(go.Scatter(x=bat.xout_list[:,n], y=bat.yout_list[:,n], name='Efficiency', marker_color='blue'))
fig5.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])), y=np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n], [bat.xin_list[-1,n]])),np.concatenate((bat.yin_list[:,n], [bat.yout_list[-1,n]])))],
                          textposition ='bottom center', name='Operation', marker_color='green'))
fig5.update_xaxes(title_text="x, %")#, range=[0.5, 2])
fig5.update_yaxes(title_text="y, %")#, range=[0.5, 2])
fig5.update_layout(title='Operation Data', width=1200, height=600,)
fig5.show()

In [ ]:
fig_equil2 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil2.add_trace(go.Scatter3d(x=data2.H3PO4_aq, y=data2.SO4_aq, z=data2.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil2.add_trace(go.Scatter3d(x=data2.H3PO4_aq, y=data2.SO4_aq, z=data2.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil2.add_trace(go.Scatter3d(x=data2.H3PO4_aq, y=data2.SO4_aq, z=data2.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil2.add_trace(go.Scatter3d(x=data2.H3PO4_aq, y=data2.SO4_aq, z=data2.SO4_org, mode='markers', name='SO4_org'), 2, 2)

bat=bat5_1
# H3PO4_org
fig_equil2.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', name='Efficiency'), 1, 1)
fig_equil2.add_trace(go.Scatter3d(x=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])), y=np.concatenate((bat.xout_list[:,2], [bat.xin_list[-1,2]])), z=np.concatenate((bat.yin_list[:,1], [bat.yout_list[-1,1]])),
                                mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),
                                                                                                                            np.concatenate((bat.xout_list[:,2], [bat.xin_list[-1,2]])),
                                                                                                                            np.concatenate((bat.yin_list[:,1], [bat.yout_list[-1,1]])))],
                                textposition ='bottom center', name='Operation', marker_color='green'),1,1)

fig_equil2.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil2.update_layout(
    scene1_zaxis_range=[0,15],
    scene2_zaxis_range=[8,10],
    scene3_zaxis_range=[0,3],
    scene4_zaxis_range=[.05,.1],
    )

fig_equil2.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))

fig_equil2.show()

## Rotem washing LLC tests

### data

In [ ]:
data=CopyPasteToArray('''
H3PO4_aq	SO4_aq	H3PO4_org	water_org	solvent_aq	SO4_org	water_org-original
0.011	0.0932	0.072	8.69	1.70	0.0186	12.09
10.325	0.0017	0.229	10.66	1.52	0.02	10.66
10.380	0.0990	0.269	8.31	1.50	0.0208	10.41
19.821	0.0031	0.635	11.3	1.34	0.01884	11.3
19.710	0.0928	0.773	10.36	1.32	0.0195	12.34
28.710	0.0057	1.435	11.96	1.28	0.01978	11.96
27.882	0.0858	1.325	9.7	1.34	0.0222	11.1
35.611	0.0071	2.871	11.94	1.28	0.036	11.94
35.887	0.0846	2.981	12.02	1.24	0.0374	12.6
43.065	0.0082	5.328	12.12	1.28	0.057	12.12
43.065	0.0802	5.604	10.38	1.06	0.0608	11.7
49.966	0.0074	8.723	12.28	1.28	0.0872	12.28
0.017	0.1540	0.042	11.48	1.46	0.0164	13.28
4.831	0.0020	0.146	12.7	1.78	0.01302	12.6
4.941	0.1078	0.173	12.06	1.66	0.0139	11.84
19.600	0.0033	0.828	11.84	1.38	0.01688	11.42
19.766	0.1120	0.856	12.46	1.38	0.01832	11.2
''')
# data
# data2=pd.DataFrame(data, columns=['H3PO4_aq',	'SO4_aq',	'H3PO4_org',	'water_org',	'solvent_aq',	'SO4_org'])
data3=pd.DataFrame(data[1:], columns=data[0])
data3_names=CopyPasteToArray('''
wp12	wp21	wp22	wp31	wp32	wp41	wp42	wp51	wp52	wp61	wp62	wp71	wp13	ww11	ww12	ww21	ww22
''')

In [ ]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh3 = x_mesh.flatten().reshape(31*31,1)
y_mesh3 = y_mesh.flatten().reshape(31*31,1)
mesh3=np.concatenate((x_mesh3, y_mesh3), axis=1)
# x_mesh[:5]

In [ ]:
a=np.concatenate((np.zeros([len(mesh3),1]),mesh3), axis=1)

In [ ]:
mesh_predict_x=np.array([EQUIL_rotem_washing3([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3([row]).yout for row in a])
mesh_predics3_1=mesh_predict_y[:,:,1]
mesh_predics3_2=mesh_predict_y[:,:,0]
mesh_predics3_3=mesh_predict_x[:,:,0]
mesh_predics3_4=mesh_predict_y[:,:,2]

### test

In [ ]:
EQUIL_rotem_washing3([[2.2,10.38,.099]]).yout

In [ ]:
test1 = LLC.Stage(90, 29, [10, 14, .07], [0, 0.1, .02], EQUIL_rotem_washing3, eff=.8)#, entrainment_perc=[1,2,3,4], entrainment_comp_in=[[10,10],[5,5]], convergence=1E-4)
test1.runs

In [ ]:
test1.output()

### 6 - Washing1 - water_org model

In [ ]:
bat6_1=LLC.Battery(stages_num=6, Oin=90, Ain=29, yin=[10, 14, .07], xin=[0, 0, 0], EQUIL=EQUIL_rotem_washing3, eff=[1,0.8,0], convergence=1E-4)
bat6_1.runs

In [ ]:
table6_1=LLC.BatteryTable(bat6_1).data

In [ ]:
# @markdown plots
fig_equil6_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil6_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil6_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil6_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil6_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil6_1.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics3_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil6_1.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics3_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil6_1.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics3_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil6_1.add_trace(go.Scatter3d(x=x_mesh3[:,0], y=y_mesh3[:,0], z=mesh_predics3_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat6_1
# n, row, col =[1,0,2], [1,1,2], [1,2,2] # H3PO4_org, water_org, SO4_org
# for i in range(3):
#   fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,n[i]], mode='markers', marker_color='blue', name='Efficiency'), row[i], col[i])
#   x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,n[i]],[bat.yout_list[-1,n[i]]]))
#   fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),row[i],col[i])

# H3PO4_org
fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil6_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil6_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil6_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil6_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil6_1.update_scenes(camera_projection_type="orthographic")

fig_equil6_1.show()

### 7 - Washing2 - water_org=12

In [ ]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh4 = x_mesh.flatten().reshape(31*31,1)
y_mesh4 = y_mesh.flatten().reshape(31*31,1)
mesh4=np.concatenate((x_mesh4, y_mesh4), axis=1)

In [ ]:
a=np.concatenate((np.zeros([len(mesh4),1]),mesh4), axis=1)

In [ ]:
mesh_predict_x=np.array([EQUIL_rotem_washing3_2([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3_2([row]).yout for row in a])
mesh_predics4_1=mesh_predict_y[:,:,1]
mesh_predics4_2=mesh_predict_y[:,:,0]
mesh_predics4_3=mesh_predict_x[:,:,0]
mesh_predics4_4=mesh_predict_y[:,:,2]

In [ ]:
bat7_1=LLC.Battery(stages_num=6, Oin=90, Ain=29, yin=[10, 14, .07], xin=[0, 0.1, .02], EQUIL=EQUIL_rotem_washing3_2, eff=.7)
bat7_1.runs

In [ ]:
table7_1=LLC.BatteryTable(bat7_1).data

In [ ]:
fig_equil7_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil7_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil7_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil7_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil7_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil7_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil7_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil7_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil7_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat7_1
# n, row, col =[1,0,2], [1,1,2], [1,2,2] # H3PO4_org, water_org, SO4_org
# for i in range(3):
#   fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,n[i]], mode='markers', marker_color='blue', name='Efficiency'), row[i], col[i])
#   x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,n[i]],[bat.yout_list[-1,n[i]]]))
#   fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),row[i],col[i])

# H3PO4_org
fig_equil7_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil7_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil7_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil7_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil7_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil7_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil7_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil7_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil7_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil7_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil7_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil7_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil7_1.update_scenes(camera_projection_type="orthographic")

fig_equil7_1.show()

### 8 - Washing3 - compare with plant data

Profile data

In [ ]:
data=CopyPasteToArray('''
x	y
0	0.0531
3.01	0.26
10.8	1.22
20.69	6.02
33.12	7.77
34.9	8.86
35.81	10.3
''')
profile_data1=pd.DataFrame(data[1:], columns=data[0])

Plant data

In [ ]:
PS_P2O5=11.6
WS_P2O5=0.2
W_flow=23.4
Prod_flow=48.4 # =36.7*1.32
Prod_P2O5=37.6

PS_flow=(Prod_flow*Prod_P2O5+WS_P2O5*(W_flow-Prod_flow))/(PS_P2O5-WS_P2O5)
WS_flow=PS_flow+W_flow-Prod_flow
P2O5_recovery=Prod_flow*Prod_P2O5/(PS_flow*PS_P2O5)
print ('Purified solvent flow =',f"{PS_flow:.1f}",'ton/hr,  ', 'Washed solvent flow =',f"{WS_flow:.1f}",'ton/hr  ', 'P2O5 recovery =',f"{P2O5_recovery:.1%}")

In [ ]:
summary_table8=pd.DataFrame([PS_flow,PS_P2O5,WS_flow,WS_P2O5,W_flow,Prod_flow,Prod_P2O5,f"{P2O5_recovery:.1%}",6,'---', '---'],
                        index=['PS_flow','PS_P2O5','WS_flow','WS_P2O5','W_flow','Prod_flow','Prod_P2O5','P2O5_recovery','Stages_num','Efficiency','Equilibrium'],
                        columns=['Plant data'])
summary_table8

[0.76853034 3.57952069] root,
[0.75617284 3.26131687] direct,
[0.77263374 3.61111111] direct *10,
[0.77057613 3.48536808 2.34910837] direct entrain

In [ ]:
bat8_1=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=.77, convergence=1E-3, entrainment_perc_out=[3.6,0])
bat8_1.runs

In [ ]:
bat=bat8_1
summary_table8['Model data-0.75-3']=[bat.Oin,bat.yin[1]/98*71,bat.Oout,bat.yout[1]/98*71,bat.Ain,bat.Aout,bat.xout[1]/98*71,f"{bat.Aout*bat.xout[1]/(bat.Oin*bat.yin[1]):.1%}", f"{bat.stages_num:.0f}", bat.eff_list[0,1], str(bat.EQUIL)[17:-2]]
summary_table8

In [ ]:
table8_1=LLC.BatteryTable(bat8_1).data

In [ ]:
## P2O5
bat=bat8_1
summary_table=summary_table8
fig8_1 = go.Figure()
n=1 #H3PO4
fig8_1.add_trace(go.Scatter(x=[0,summary_table['Plant data'].loc['Prod_P2O5']], y=[summary_table['Plant data'].loc['WS_P2O5'],summary_table['Plant data'].loc['PS_P2O5']], name='Plant data', marker_color='orange'))
fig8_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='Model data', marker_color='green'))
fig8_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
fig8_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))
fig8_1.add_trace(go.Scatter(x=profile_data1.x, y=profile_data1.y, name='Plant profile data', marker_color='grey', visible='legendonly'))

fig8_1.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig8_1.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
# fig8_1.update_layout(title='', width=1200, height=600,)
fig8_1.show()

In [ ]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh4 = x_mesh.flatten().reshape(31*31,1)
y_mesh4 = y_mesh.flatten().reshape(31*31,1)
mesh4=np.concatenate((x_mesh4, y_mesh4), axis=1)

In [ ]:
a=np.concatenate((np.zeros([len(mesh4),1]),mesh4), axis=1)

In [ ]:
mesh_predict_x=np.array([EQUIL_rotem_washing3_3([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3_3([row]).yout for row in a])
mesh_predics4_1=mesh_predict_y[:,:,1]
mesh_predics4_2=mesh_predict_y[:,:,0]
mesh_predics4_3=mesh_predict_x[:,:,0]
mesh_predics4_4=mesh_predict_y[:,:,2]

In [ ]:
fig_equil8_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil8_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil8_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil8_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil8_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil8_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil8_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil8_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil8_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat8_1
# n, row, col =[1,0,2], [1,1,2], [1,2,2] # H3PO4_org, water_org, SO4_org
# for i in range(3):
#   fig_equil6_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,n[i]], mode='markers', marker_color='blue', name='Efficiency'), row[i], col[i])
#   x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,n[i]],[bat.yout_list[-1,n[i]]]))
#   fig_equil6_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),row[i],col[i])

# H3PO4_org
fig_equil8_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil8_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil8_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil8_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil8_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil8_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil8_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil8_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil8_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil8_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil8_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil8_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil8_1.update_scenes(camera_projection_type="orthographic")

fig_equil8_1.show()

### 10 - Washing5 - optimize.direct()

Plant data

In [ ]:
PS_P2O5=11.6
WS_P2O5=0.2
W_flow=23.4
Prod_flow=48.4 # =36.7*1.32
Prod_P2O5=37.6

PS_flow=(Prod_flow*Prod_P2O5+WS_P2O5*(W_flow-Prod_flow))/(PS_P2O5-WS_P2O5)
WS_flow=PS_flow+W_flow-Prod_flow
P2O5_recovery=Prod_flow*Prod_P2O5/(PS_flow*PS_P2O5)
print ('Purified solvent flow =',f"{PS_flow:.1f}",'ton/hr,  ', 'Washed solvent flow =',f"{WS_flow:.1f}",'ton/hr  ', 'P2O5 recovery =',f"{P2O5_recovery:.1%}")

In [ ]:
summary_table10=pd.DataFrame([PS_flow,PS_P2O5,WS_flow,WS_P2O5,W_flow,Prod_flow,Prod_P2O5,f"{P2O5_recovery:.1%}",6,'---', '---'],
                        index=['PS_flow','PS_P2O5','WS_flow','WS_P2O5','W_flow','Prod_flow','Prod_P2O5','P2O5_recovery','Stages_num','Efficiency','Equilibrium'],
                        columns=['Plant data'])
summary_table10

#### slope(entr,eff)

נמצא שהתלות של השיפוע של קו הפעולה תלוי באחוז סחיפות בקשר ליניארי וביעילות בקשר
מרובע. מחשבים פרמטרים של פונקציה זאת

slope = a + b * entr + c * eff + d * eff^2

בשלב ראשון מוסיפים עמודה עם יעילות בריבוע

מתוך הריכוזים של התהליך מחשבים שיפוע של התהליך. מוצאים קשר בין סחיפות ויעילות.

entrain=(slope_proc-C[0]-C[2]*eff-C[3]*eff**2)/C[1]

האופטימיזציה הופכת לאחת עם משתנה בודד.

In [ ]:
## P2O5
summary_table=summary_table10
fig10_1 = go.Figure()
n=1 #H3PO4

for entr in [2.6, 3.6, 4.6]:
  for eff in [0.67,0.77,0.87]:
    bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff, convergence=1E-3, entrainment_perc_out=[entr,0])
    print ('entr:', entr, '  eff:', eff, '  slope:', (bat.yin[1]-bat.yout[1])/(bat.xout[1]-bat.xin[1]))
    fig10_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          # text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='entr:'+str(entr)+', '+'eff:'+str(eff)))

bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=.77, convergence=1E-3, entrainment_perc_out=[3.6,0])
fig10_1.add_trace(go.Scatter(x=[0,summary_table['Plant data'].loc['Prod_P2O5']], y=[summary_table['Plant data'].loc['WS_P2O5'],summary_table['Plant data'].loc['PS_P2O5']], name='Plant data', marker_color='orange'))
fig10_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='Model data', marker_color='green'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))

fig10_1.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig10_1.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
# fig10_1.update_layout(title='', width=1200, height=600,)
fig10_1.show()

In [ ]:
slopes=[]
for entr in [3.6, 2.6, 4.6]: # לינארי -- מספיק 2
  for eff in [0.77,0.67,0.87]: # פורבולה -- צריך 3
    bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff, convergence=1E-3, entrainment_perc_out=[entr,0])
    slope=(bat.yin[1]-bat.yout[1])/(bat.xout[1]-bat.xin[1]) # שיפוע של קו הפעולה
    slopes.append([entr, eff, slope])
    # print ('entr:', entr, '  eff:', eff, '  slope:', slope)
arr=np.array(slopes)
arr

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=arr[:,0], y=arr[:,1], z=arr[:,2], mode = "markers"))
fig.show()

In [ ]:
arr_mod=np.c_[np.ones([len(arr),1]),arr[:,0:2], arr[:,1]**2, arr[:,2]]
arr_mod

In [ ]:
rows=[0,1,2,3]
A=arr_mod[rows,0:4]
B=arr_mod[rows,4]
C=np.dot(np.linalg.inv(A),B) # מחשבים ערכים של עמודה אחרונה (קו הפעולה) מתוך עמודות ראשונות
C

In [ ]:
pred=np.dot(arr_mod[:,0:4],C)
pred

In [ ]:
fig=go.Figure()
fig.add_trace(go.Scatter3d(x=arr[:,0], y=arr[:,1], z=arr[:,2], mode = "markers"))
# fig.add_trace(go.Scatter3d(x=arr[rows,0], y=arr[rows,1], z=pred[rows], mode = "markers"))
fig.add_trace(go.Scatter3d(x=arr[:,0], y=arr[:,1], z=pred, mode = "markers"))

fig.show()

#### together

Plant data

In [ ]:
PS_P2O5=11.6
WS_P2O5=0.2
W_flow=23.4
Prod_flow=48.4 # =36.7*1.32
Prod_P2O5=37.6
PS_flow=(Prod_flow*Prod_P2O5+WS_P2O5*(W_flow-Prod_flow))/(PS_P2O5-WS_P2O5)
WS_flow=PS_flow+W_flow-Prod_flow
P2O5_recovery=Prod_flow*Prod_P2O5/(PS_flow*PS_P2O5)
print ('Purified solvent flow =',f"{PS_flow:.1f}",'ton/hr,  ', 'Washed solvent flow =',f"{WS_flow:.1f}",'ton/hr  ', 'P2O5 recovery =',f"{P2O5_recovery:.1%}")

slope_proc=(PS_P2O5-WS_P2O5)/Prod_P2O5

In [ ]:
summary_table10_1=pd.DataFrame([PS_flow,PS_P2O5,WS_flow,WS_P2O5,W_flow,Prod_flow,Prod_P2O5,f"{P2O5_recovery:.1%}",6,'---', '---'],
                        index=['PS_flow','PS_P2O5','WS_flow','WS_P2O5','W_flow','Prod_flow','Prod_P2O5','P2O5_recovery','Stages_num','Efficiency','Equilibrium'],
                        columns=['Plant data'])
summary_table10_1

Slope(entrain,eff)

נמצא שהתלות של השיפוע של קו הפעולה תלוי באחוז סחיפות בקשר ליניארי וביעילות בקשר
מרובע. מחשבים פרמטרים של פונקציה זאת

slope = a + b * entr + c * eff + d * eff^2

בשלב ראשון מוסיפים עמודה עם יעילות בריבוע

מתוך הריכוזים של התהליך מחשבים שיפוע של התהליך. מוצאים קשר בין סחיפות ויעילות.

entrain=(slope_proc-C[0]-C[2]*eff-C[3]*eff**2)/C[1]

האופטימיזציה הופכת לאחת עם משתנה בודד.

In [ ]:
slopes=[]
entr_initial, eff_init = 3.6, 0.77
for entr in [entr_initial, entr_initial*.75]: # לינארי -- מספיק 2
  for eff in [eff_init,eff_init*.85,eff_init*1.1]: # פורבולה -- צריך 3
    bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff, convergence=1E-3, entrainment_perc_out=[entr,0])
    slope=(bat.yin[1]-bat.yout[1])/(bat.xout[1]-bat.xin[1])
    slopes.append([entr, eff, slope])
    # print ('entr:', entr, '  eff:', eff, '  slope:', slope)
arr=np.array(slopes)
arr_mod=np.c_[np.ones([len(arr),1]),arr[:,0:2], arr[:,1]**2, arr[:,2]] # entrainment approximated to porabola

In [ ]:
rows=[0,1,2,3] # 4 rows for 4 unknowns
A=arr_mod[rows,0:4]
B=arr_mod[rows,4]
C=np.dot(np.linalg.inv(A),B) # פתרון מערכת משוואות.  מחשבים ערכים של עמודה אחרונה (קו הפעולה) מתוך עמודות ראשונות

optimize

optimization with 1 parameter

במקום להריץ אופטימיזציה עם 2 משתנים: יעילות וסחיפות, זה רץ על פרמטר בודד -- יעילות. הסחיפות מחושבות מתוך תלות בין סחיפות ושיפוע קו פעולה. תלות זאת חושבה לפני זה.

In [ ]:
def P2O5_calc3_1(x):
  eff = x[0]
  entrain=(slope_proc-C[0]-C[2]*eff-C[3]*eff**2)/C[1]
  bat=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff, convergence=1E-3, entrainment_perc_out=[entrain,0])
  # error1=abs((WS_P2O5-bat.yout[1]/98*71)/WS_P2O5)
  error=abs((Prod_P2O5-bat.xout[1]/98*71)/Prod_P2O5)
  print (eff, entrain, bat.yout[1]/98*71, bat.xout[1]/98*71, error, bat.runs, ',')

  return error

In [ ]:
bounds=[(0.5,1)]
sol10_1=optimize.direct(P2O5_calc3_1, bounds=bounds, f_min=0, f_min_rtol=0.001, maxiter=100, locally_biased=False)
print (sol10_1.x, sol10_1.fun)

In [ ]:
eff10_1=sol10_1.x[0]
entrain10_1=entrain=(slope_proc-C[0]-C[2]*eff10_1-C[3]*eff10_1**2)/C[1]

optimal calcul

In [ ]:
bat10_1=LLC.Battery(stages_num=6, Oin=PS_flow, Ain=W_flow, yin=[12,PS_P2O5/71*98,.07], xin=[0,0,.02], EQUIL=EQUIL_rotem_washing3_3, eff=eff10_1, convergence=1E-3, entrainment_perc_out=[entrain10_1,0])
bat10_1.runs

In [ ]:
bat=bat10_1
summary_table10_1['Model data']=[bat.Oin,bat.yin[1]/98*71,bat.Oout,bat.yout[1]/98*71,bat.Ain,bat.Aout,bat.xout[1]/98*71,f"{bat.Aout*bat.xout[1]/(bat.Oin*bat.yin[1]):.1%}", f"{bat.stages_num:.0f}", bat.eff_list[0,1], str(bat.EQUIL)[17:-2]]
summary_table10_1

In [ ]:
table10_1=LLC.BatteryTable(bat10_1).data

#### vizual

In [ ]:
## P2O5
bat=bat10_1
summary_table=summary_table10_1
fig10_1 = go.Figure()
n=1 #H3PO4
fig10_1.add_trace(go.Scatter(x=[0,summary_table['Plant data'].loc['Prod_P2O5']], y=[summary_table['Plant data'].loc['WS_P2O5'],summary_table['Plant data'].loc['PS_P2O5']], name='Plant data', marker_color='orange'))
fig10_1.add_trace(go.Scatter(x=np.concatenate((bat.xout_list[:,n]*71/98, [bat.xin_list[-1,n]/98*71])), y=np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])), mode = "markers+lines+text",
                          text= ['({:.2f},{:.2f})'.format(a,b) for a,b in zip(np.concatenate((bat.xout_list[:,n]/98*71, [bat.xin_list[-1,n]/98*71])),np.concatenate((bat.yin_list[:,n]/98*71, [bat.yout_list[-1,n]/98*71])))],
                          textposition ='bottom center', name='Model data', marker_color='green'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_list[:,n]/98*71, name='Efficiency', marker_color='blue'))
fig10_1.add_trace(go.Scatter(x=bat.xout_list[:,n]/98*71, y=bat.yout_tag_list[:,n]/98*71, name='Equilibrium', marker_color='black'))
fig10_1.add_trace(go.Scatter(x=profile_data1.x, y=profile_data1.y, name='Plant profile data', marker_color='grey', visible='legendonly'))

fig10_1.update_xaxes(title_text="%P2O5 aq")#, range=[0.5, 2])
fig10_1.update_yaxes(title_text="%P2O5 org")#, range=[0.5, 2])
# fig10_1.update_layout(title='', width=1200, height=600,)
fig10_1.show()

In [ ]:
# x_mesh, y_mesh = np.linspace(0, 60, 31), np.linspace(0, 0.2, 31) # expanded x and y extrapolation area observation
x_mesh, y_mesh = np.linspace(0, 50, 31), np.linspace(0, 0.15, 31)
x_mesh, y_mesh = np.meshgrid(x_mesh, y_mesh)
x_mesh4 = x_mesh.flatten().reshape(31*31,1)
y_mesh4 = y_mesh.flatten().reshape(31*31,1)
mesh4=np.concatenate((x_mesh4, y_mesh4), axis=1)

In [ ]:
a=np.concatenate((np.zeros([len(mesh4),1]),mesh4), axis=1)

In [ ]:
mesh_predict_x=np.array([EQUIL_rotem_washing3_3([row]).xout for row in a])
mesh_predict_y=np.array([EQUIL_rotem_washing3_3([row]).yout for row in a])
mesh_predics4_1=mesh_predict_y[:,:,1]
mesh_predics4_2=mesh_predict_y[:,:,0]
mesh_predics4_3=mesh_predict_x[:,:,0]
mesh_predics4_4=mesh_predict_y[:,:,2]

In [ ]:
fig_equil10_1 = make_subplots(rows=2, cols=2, subplot_titles=('H3PO4_org','water_org','solvent_aq','SO4_org'), column_widths=[0.5, 0.5], vertical_spacing=0.05,
                     specs=[[{"type": "scene"}, {"type": "scene"}],[{"type": "scene"}, {"type": "scene"}]])

fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.H3PO4_org, mode='markers', name='H3PO4_org'), 1, 1)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.water_org, mode='markers', name='water_org'), 1, 2)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.solvent_aq, mode='markers', name='solvent_aq'), 2, 1)
fig_equil10_1.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='SO4_org'), 2, 2)

# visib='legendonly'
visib=True
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_1[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,1)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_2[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),1,2)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_3[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,1)
fig_equil10_1.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0], mode='markers',marker_color='green', marker_size=0.5, name='Equilibrium surface', visible=visib),2,2)

bat=bat10_1
# H3PO4_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,1], mode='markers', marker_color='blue', name='Efficiency'), 1, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,1],[bat.yout_list[-1,1]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,1)
# water_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 1, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,0],[bat.yout_list[-1,0]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),1,2)
# SO4_org
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.yout_list[:,2], mode='markers', marker_color='blue', name='Efficiency'), 2, 2)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.yin_list[:,2],[bat.yout_list[-1,2]]))
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,2)

# solvent_aq
fig_equil10_1.add_trace(go.Scatter3d(x=bat.xout_list[:,1], y=bat.xout_list[:,2], z=bat.xout_list[:,0], mode='markers', marker_color='blue', name='Efficiency'), 2, 1)
x,y,z=np.concatenate((bat.xout_list[:,1], [bat.xin_list[-1,1]])),np.concatenate((bat.xout_list[:,2],[bat.xin_list[-1,2]])), np.concatenate((bat.xout_list[:,0],[bat.xin_list[-1,0]])) # aqueous -- different operation line order
fig_equil10_1.add_trace(go.Scatter3d(x=x, y=y, z=z, mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)], textposition ='bottom center', name='Operation', marker_color='green'),2,1)


fig_equil10_1.update_layout(title='', autosize=True,
                  width=980, height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig_equil10_1.update_layout(
    # scene1_zaxis_range=[0,10],
    # scene2_zaxis_range=[10,14],
    # scene3_zaxis_range=[0,3],
    # scene4_zaxis_range=[0,.1],
    )

# fig_equil10_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=1.5, y=-1.5, z=1.5))
fig_equil10_1.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig_equil10_1.update_scenes(camera_projection_type="orthographic")

fig_equil10_1.show()

# Profile

In [ ]:
data22=CopyPasteToArray('''
H3PO4_aq	SO4_aq	SO4_org	Unit
4.15	0.147	0.0626	571
14.91	0.0806	0.0852	573
28.56	0.0329	0.0787	575
45.71	0.0106	0.0664	579
48.17	0.0078	0.0631	581
49.43	0.0058	0.0641	5150
''')

data22=pd.DataFrame(data22[1:], columns=data22[0])

In [ ]:
data22

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter3d(x=data3.H3PO4_aq, y=data3.SO4_aq, z=data3.SO4_org, mode='markers', name='Tests'))
fig.add_trace(go.Scatter3d(x=x_mesh4[:,0], y=y_mesh4[:,0], z=mesh_predics4_4[:,0],
                                     mode='markers',marker_color='gray', marker_size=1, name='Equilibrium surface', visible=True))
fig.add_trace(go.Scatter3d(x=data22.H3PO4_aq, y=data22.SO4_aq, z=data22.SO4_org,
                                     mode = "markers+lines+text", text= ['({:.2f},{:.2f},{:.2f})'.format(a,b,c) for a,b,c in zip(x,y,z)],
                                     textposition ='bottom center', name='Profile', marker_color='green'))

fig.update_layout(title='', autosize=True,
                  # width=980,
                  height=980,
                  margin=dict(l=0, r=0, b=0, t=30),
                  )

fig.update_scenes(xaxis_title='H3PO4_aq', yaxis_title='SO4_aq',  zaxis_title='SO4_org', camera_eye=dict(x=0, y=-2.2, z=0.01))
fig.update_scenes(camera_projection_type="orthographic")

# fig.write_html("Profile_SO4.html")
fig.show()